# Eyettention

In [ ]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import model
import torch
from torch.utils import model_zoo
import pandas as pd
from utils import *
from sklearn.model_selection import StratifiedKFold, KFold
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam, RMSprop
from transformers import BertTokenizerFast
from model import Eyettention
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from torch.nn.functional import cross_entropy, softmax
from collections import deque
import pickle
import json
import matplotlib.pyplot as plt
import argparse

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
DEVICE = 'cuda'
#DEVICE = 'cpu'
#test_mode = 'text'
test_mode = 'subject'
scanpath_gen_flag = True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Loading the data**

In [ ]:
#load corpus
word_info_df, pos_info_df, eyemovement_df = load_corpus(cf["dataset"])

In [ ]:
word_info_df.head()

,SN,NW,WORD,POS,LEN,STRK,WF_BLI,WF_SUB,PRED,CF1_BLI,CF2_BLI,CF1_SUB,CF2_SUB,C1_STRK,C2_STRK,C3_STRK
0,1,1,我,r,1,7,13642.939135,50147.83,-1.055107,11773.956446,NaN,43956.70,NaN,7,NaN,NaN
1,1,2,怀念,v,2,15,20.977519,17.71,-2.145230,189.167322,376.625301,186.29,160.76,7,8.0,NaN
2,1,3,那些,r,2,14,101.531191,788.46,-1.213874,3130.377309,1484.849521,7792.81,2415.02,6,8.0,NaN
3,1,4,日子,n,2,7,201.384181,93.48,-1.431100,1227.308468,3884.767472,494.25,3793.53,4,3.0,NaN
4,1,5,里,n,1,7,2623.028952,1886.22,-0.430101,2760.589382,NaN,4071.08,NaN,7,NaN,NaN


In [ ]:
pos_info_df.head()

,0,1
0,n,noun
1,v,verb
2,a,adjective
3,m,numeral
4,q,quantifier


In [ ]:
eyemovement_df.head(n=50)

,id,sn,nw,wn,fl,dur,ao,dir,o,l,...,wnx2,flx2,durx2,aox2,dirx2,ox2,lx2,fx2,px2,ix2
0,1,136,11,1,0.558073,96,7.705933,1,NaN,0.500000,...,5.0,1.264006,166.0,-4.776237,-1.0,7.705933,0.500000,0.546235,-2.167317,21.0
1,1,136,11,5,1.264006,166,-4.776237,-1,7.705933,0.500000,...,3.0,0.487769,348.0,2.395043,1.0,-4.776237,0.500000,1.725016,-2.167317,16.0
2,1,136,11,3,0.487769,348,2.395043,1,-4.776237,0.500000,...,4.0,0.882812,124.0,3.388304,1.0,2.395043,0.500000,1.418835,-0.858671,16.0
3,1,136,11,4,0.882812,124,3.388304,1,2.395043,0.500000,...,6.0,0.271116,206.0,3.376014,1.0,3.388304,0.500000,1.757428,-0.916454,11.0
4,1,136,11,6,0.271116,206,3.376014,1,3.388304,0.500000,...,8.0,0.647130,196.0,-844.142250,-1.0,3.376014,0.500000,0.938246,-1.243038,18.0
5,1,136,11,8,0.647130,196,-844.142250,-1,3.376014,0.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,97,12,1,0.504880,146,5.542638,1,-844.142250,0.500000,...,4.0,0.047518,204.0,-2.937383,-1.0,5.542638,0.500000,1.980993,-2.167317,15.0
7,1,97,12,4,0.047518,204,-2.937383,-1,5.542638,0.500000,...,2.0,1.110135,148.0,2.797583,1.0,-2.937383,0.500000,0.865288,-1.863323,11.0
8,1,97,12,2,1.110135,148,2.797583,1,-2.937383,0.500000,...,3.0,1.907718,132.0,3.219782,1.0,2.797583,0.500000,1.923677,-1.556303,21.0
9,1,97,12,3,1.907718,132,3.219782,1,2.797583,0.500000,...,5.0,1.127500,120.0,6.258535,1.0,3.219782,0.500000,1.656767,-1.556303,12.0


In [ ]:
#Make list with sentence index
sn_list = np.unique(eyemovement_df.sn.values).tolist()

In [ ]:
len(sn_list) # 150 unique sentences in the dataset

150

In [ ]:
#Make list with reader index
reader_list = np.unique(eyemovement_df.id.values).tolist()

In [ ]:
len(reader_list) # 60 unique readers in the dataset

60

In [ ]:
save_data_folder = "./drive/MyDrive/results/BSC"
atten_type = "local_g"
#test_mode = 'text'
test_mode = 'subject'

**Training loop**

In [4]:
import numpy as np
import pandas as pd
import os
from utils import *
from sklearn.model_selection import StratifiedKFold, KFold
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam, RMSprop
from transformers import BertTokenizer
from model import Eyettention
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from torch.nn.functional import cross_entropy, softmax
from collections import deque
import pickle
import json
import matplotlib.pyplot as plt
import argparse

if __name__ == '__main__':
	gpu = 0

	torch.set_default_tensor_type('torch.FloatTensor')
	availbl = torch.cuda.is_available()
	if availbl:
		device = f'cuda:{gpu}'
	else:
		device = 'cpu'
	torch.cuda.set_device(gpu)

	cf = {"model_pretrained": "bert-base-chinese",
			"lr": 1e-3,
			"max_grad_norm": 10,
			"n_epochs": 20,  # 1000
			"n_folds": 5,
			"dataset": 'BSC',
			"atten_type": 'local-g',
			"batch_size": 256,
			"max_sn_len": 27, #include start token and end token
			"max_sp_len": 40, #include start token and end token
			"norm_type": "z-score",
			"earlystop_patience": 20,
			"max_pred_len": 60
			}

	#Encode the label into interger categories, setting the exclusive category 'cf["max_sn_len"]-1' as the end sign
	le = LabelEncoder()
	le.fit(np.append(np.arange(-cf["max_sn_len"]+3, cf["max_sn_len"]-1), cf["max_sn_len"]-1))
	#le.classes_

	#load corpus
	word_info_df, pos_info_df, eyemovement_df = load_corpus(cf["dataset"])
	#Make list with sentence index
	sn_list = np.unique(eyemovement_df.sn.values).tolist()
	#Make list with reader index
	reader_list = np.unique(eyemovement_df.id.values).tolist()

	#Split training&test sets by text or reader, depending on configuration
	if test_mode == 'text':
		print('Start evaluating on new sentences.')
		split_list = sn_list
	elif test_mode == 'subject':
		print('Start evaluating on new readers.')
		split_list = reader_list

	n_folds = cf["n_folds"]
	kf = KFold(n_splits=n_folds, shuffle=True, random_state=0)
	fold_indx = 0
	#for scanpath generation
	sp_dnn_list = []
	sp_human_list = []
	for train_idx, test_idx in kf.split(split_list):
		loss_dict = {'val_loss':[], 'train_loss':[], 'test_ll':[]}
		list_train = [split_list[i] for i in train_idx]
		list_test = [split_list[i] for i in test_idx]

		# create train validation split for training the models:
		kf_val = KFold(n_splits=n_folds, shuffle=True, random_state=0)
		for train_index, val_index in kf_val.split(list_train):
			# we only evaluate a single fold
			break
		list_train_net = [list_train[i] for i in train_index]
		list_val_net = [list_train[i] for i in val_index]

		if test_mode == 'text':
			sn_list_train = list_train_net
			sn_list_val = list_val_net
			sn_list_test = list_test
			reader_list_train, reader_list_val, reader_list_test = reader_list, reader_list, reader_list

		elif test_mode == 'subject':
			reader_list_train = list_train_net
			reader_list_val = list_val_net
			reader_list_test = list_test
			sn_list_train, sn_list_val, sn_list_test = sn_list, sn_list, sn_list

		#initialize tokenizer
		tokenizer = BertTokenizer.from_pretrained(cf['model_pretrained'])
		#Preparing batch data
		dataset_train = BSCdataset(word_info_df, eyemovement_df, cf, reader_list_train, sn_list_train, tokenizer)
		train_dataloaderr = DataLoader(dataset_train, batch_size = cf["batch_size"], shuffle = True, drop_last=True)

		dataset_val = BSCdataset(word_info_df, eyemovement_df, cf, reader_list_val, sn_list_val, tokenizer)
		val_dataloaderr = DataLoader(dataset_val, batch_size = cf["batch_size"], shuffle = False, drop_last=True)

		dataset_test = BSCdataset(word_info_df, eyemovement_df, cf, reader_list_test, sn_list_test, tokenizer)
		test_dataloaderr = DataLoader(dataset_test, batch_size = cf["batch_size"], shuffle = False, drop_last=False)

		#z-score normalization for gaze features
		fix_dur_mean, fix_dur_std = calculate_mean_std(dataloader=train_dataloaderr, feat_key="sp_fix_dur", padding_value=0, scale=1000)
		landing_pos_mean, landing_pos_std = calculate_mean_std(dataloader=train_dataloaderr, feat_key="sp_landing_pos", padding_value=0)
		sn_word_len_mean, sn_word_len_std = calculate_mean_std(dataloader=train_dataloaderr, feat_key="sn_word_len")

		# load model
		dnn = Eyettention(cf)

		#training
		episode = 0
		optimizer = Adam(dnn.parameters(), lr=cf["lr"])
		dnn.train()
		dnn.to(device)
		av_score = deque(maxlen=100)
		av_location_score = deque(maxlen=100)
		av_duration_score = deque(maxlen=100)
		old_score = 1e10
		save_ep_couter = 0
		print('Start training')
		for episode_i in range(episode, cf["n_epochs"]+1):
			dnn.train()
			print('episode:', episode_i)
			counter = 0
			for batchh in train_dataloaderr:
				counter += 1
				batchh.keys()
				sn_ids = batchh["sn_ids"].to(device)
				sn_input_ids = batchh["sn_input_ids"].to(device)
				sn_attention_mask = batchh["sn_attention_mask"].to(device)
				sp_input_ids = batchh["sp_input_ids"].to(device)
				sp_attention_mask = batchh["sp_attention_mask"].to(device)
				sp_pos = batchh["sp_pos"].to(device)
				sp_landing_pos = batchh["sp_landing_pos"].to(device) # [256, 40]
				sp_fix_dur = (batchh["sp_fix_dur"]/1000).to(device) # [256, 40]
				sn_word_len = batchh["sn_word_len"].to(device)


				# normalize gaze features (z-score normalisation)
				mask = ~torch.eq(sp_fix_dur, 0)
				sp_fix_dur = (sp_fix_dur-fix_dur_mean)/fix_dur_std * mask
				sp_fix_dur = torch.nan_to_num(sp_fix_dur) # [256, 40]
				sp_landing_pos = (sp_landing_pos - landing_pos_mean)/landing_pos_std * mask
				sp_landing_pos = torch.nan_to_num(sp_landing_pos)
				sn_word_len = (sn_word_len - sn_word_len_mean)/sn_word_len_std
				sn_word_len = torch.nan_to_num(sn_word_len)

				# zero old gradients
				optimizer.zero_grad()
				# predict output with DNN
				location_preds, duration_preds, atten_weights = dnn(sn_emd=sn_input_ids,
											sn_mask=sn_attention_mask,
											sp_emd=sp_input_ids,
											sp_pos=sp_pos,
											word_ids_sn=None,
											word_ids_sp=None,
											sp_fix_dur=sp_fix_dur,
											sp_landing_pos=sp_landing_pos,
											sn_word_len = sn_word_len)#[batch, step, dec_o_dim]

				location_preds = location_preds.permute(0,2,1)              #[batch, dec_o_dim, step]

				#prepare label and mask
				# Compute loss for fixation locations
				pad_mask, label = load_label(sp_pos, cf, le, device)
				loss = nn.CrossEntropyLoss(reduction="none")
		#		label = torch.tensor(label, dtype=torch.long, device=device) # added because of the error message when trying to run the code
		#		pad_mask = torch.tensor(pad_mask, dtype=torch.bool, device=device) # needs to be added when running without GPU
				batch_location_error = torch.mean(torch.masked_select(loss(location_preds, label), ~pad_mask))

				# Compute loss for fixation durations
				duration_labels = sp_fix_dur[:, :39] # Adjust duration_labels to match the sequence length of duration_preds
				duration_preds = duration_preds.squeeze(-1)  # Remove extra dimension (from [256, 39, 1] to [256, 39])
				dur_mask = mask[:, :39]
				dur_loss = nn.MSELoss(reduction="none")
				# apply a mask since these tensors are padded and we don't want padded values to contribute to the loss
				batch_duration_error = torch.mean(torch.masked_select(dur_loss(duration_preds, duration_labels), dur_mask))

				# Combined loss for both location and duration
				batch_error = batch_location_error + batch_duration_error

				# backpropagate loss
				batch_error.backward()
				# clip gradients
				gradient_clipping(dnn, cf["max_grad_norm"])

				#learn
				optimizer.step()
				av_location_score.append(batch_location_error.to('cpu').detach().numpy())
				av_duration_score.append(batch_duration_error.to('cpu').detach().numpy())
				print('counter:',counter)
				print('\rSample {}\tLocation Loss: {:.10f}\tDuration Loss: {:.10f}'.format(
					counter, np.mean(av_location_score), np.mean(av_duration_score)), end=" ")

		location_val_loss = []
		duration_val_loss = []
		val_loss = []
		dnn.eval()
		for batchh in val_dataloaderr:
			with torch.no_grad():
				sn_ids_val = batchh["sn_ids"].to(device)
				sn_input_ids_val = batchh["sn_input_ids"].to(device)
				sn_attention_mask_val = batchh["sn_attention_mask"].to(device)
				sp_input_ids_val = batchh["sp_input_ids"].to(device)
				sp_attention_mask_val = batchh["sp_attention_mask"].to(device)
				sp_pos_val = batchh["sp_pos"].to(device)
				sp_landing_pos_val = batchh["sp_landing_pos"].to(device)
				sp_fix_dur_val = (batchh["sp_fix_dur"]/1000).to(device)
				sn_word_len_val = batchh["sn_word_len"].to(device)

				#normalize gaze features
				mask = ~torch.eq(sp_fix_dur_val, 0)
				sp_fix_dur_val = (sp_fix_dur_val-fix_dur_mean)/fix_dur_std * mask
				sp_landing_pos_val = (sp_landing_pos_val - landing_pos_mean)/landing_pos_std * mask
				sp_fix_dur_val = torch.nan_to_num(sp_fix_dur_val)
				sp_landing_pos_val = torch.nan_to_num(sp_landing_pos_val)
				sn_word_len_val = (sn_word_len_val - sn_word_len_mean)/sn_word_len_std
				sn_word_len_val = torch.nan_to_num(sn_word_len_val)

				location_preds_val, duration_preds_val, atten_weights_val = dnn(sn_emd=sn_input_ids_val,
														sn_mask=sn_attention_mask_val,
														sp_emd=sp_input_ids_val,
														sp_pos=sp_pos_val,
														word_ids_sn=None,
														word_ids_sp=None,
														sp_fix_dur=sp_fix_dur_val,
														sp_landing_pos=sp_landing_pos_val,
														sn_word_len = sn_word_len_val)#[batch, step, dec_o_dim]
				location_preds_val = location_preds_val.permute(0,2,1)              #[batch, dec_o_dim, step

				# Compute location prediction error
				loss = nn.CrossEntropyLoss(reduction="none")
				pad_mask_val, label_val = load_label(sp_pos_val, cf, le, device)
			#	label_val = torch.tensor(label_val, dtype=torch.long, device=device) # changed type
			#	pad_mask_val = torch.tensor(pad_mask_val, dtype=torch.bool, device=device) # changed type
				location_error_val = torch.mean(torch.masked_select(loss(location_preds_val, label_val), ~pad_mask_val))
				location_val_loss.append(location_error_val.detach().to('cpu').numpy())

				# Compute duration prediction error
				duration_labels_val = sp_fix_dur_val[:, :39] # Adjust duration_labels to match the sequence length of duration_preds
				duration_preds_val = duration_preds_val.squeeze(-1)
				dur_mask_val = mask[:, :39]
				dur_loss = nn.MSELoss(reduction="none")
				duration_error_val = torch.mean(torch.masked_select(dur_loss(duration_preds_val, duration_labels_val), dur_mask_val))
				duration_val_loss.append(duration_error_val.detach().to('cpu').numpy())

				combined_loss = location_error_val + duration_error_val
				val_loss.append(combined_loss.detach().to('cpu').numpy())

			print('\nValidation loss for locations {} \n'.format(np.mean(location_val_loss)))
			print('\nValidation loss for duration {} \n'.format(np.mean(duration_val_loss)))
			loss_dict['val_loss'].append(np.mean(val_loss))

			if np.mean(val_loss) < old_score:
				# save model if val loss is smallest
				torch.save(dnn.state_dict(), '{}/CELoss_BSC_{}_eyettention_{}_newloss_fold{}.pth'.format(save_data_folder, test_mode, atten_type, fold_indx))
				old_score = np.mean(val_loss)
				print('\nsaved model state dict\n')
				save_ep_couter = episode_i
			else:
				#early stopping
				if episode_i - save_ep_couter >= cf["earlystop_patience"]:
					break

		#evaluation
		dnn.eval()
		res_llh=[]
		res_mse = []
		dnn.load_state_dict(torch.load(os.path.join(save_data_folder, f'CELoss_BSC_{test_mode}_eyettention_{atten_type}_newloss_fold{fold_indx}.pth'), map_location='cpu'))
		dnn.to(device)
		batch_indx = 0
		for batchh in test_dataloaderr:
			with torch.no_grad():
				sn_ids_test = batchh["sn_ids"].to(device)
				sn_input_ids_test = batchh["sn_input_ids"].to(device)
				sn_attention_mask_test = batchh["sn_attention_mask"].to(device)
				sp_input_ids_test = batchh["sp_input_ids"].to(device)
				sp_attention_mask_test = batchh["sp_attention_mask"].to(device)
				sp_pos_test = batchh["sp_pos"].to(device) # 28: '<Sep>', 29: '<'Pad'>'
				sp_landing_pos_test = batchh["sp_landing_pos"].to(device)
				sp_fix_dur_test = (batchh["sp_fix_dur"]/1000).to(device)
				sn_word_len_test = batchh["sn_word_len"].to(device)

				#normalize gaze features
				mask = ~torch.eq(sp_fix_dur_test, 0)
				sp_fix_dur_test = (sp_fix_dur_test-fix_dur_mean)/fix_dur_std * mask
				sp_landing_pos_test = (sp_landing_pos_test - landing_pos_mean)/landing_pos_std * mask
				sp_fix_dur_test = torch.nan_to_num(sp_fix_dur_test)
				sp_landing_pos_test = torch.nan_to_num(sp_landing_pos_test)
				sn_word_len_test = (sn_word_len_test - sn_word_len_mean)/sn_word_len_std
				sn_word_len_test = torch.nan_to_num(sn_word_len_test)

				location_preds_test, duration_preds_test, atten_weights_test = dnn(sn_emd=sn_input_ids_test,
														sn_mask=sn_attention_mask_test,
														sp_emd=sp_input_ids_test,
														sp_pos=sp_pos_test,
														word_ids_sn=None,
														word_ids_sp=None,
														sp_fix_dur=sp_fix_dur_test,
														sp_landing_pos=sp_landing_pos_test,
														sn_word_len = sn_word_len_test) #[batch, step, dec_o_dim]


				########## Evaluate location predictions ##########
				#We do not use nn.CrossEntropyLoss here to calculate the likelihood because it combines nn.LogSoftmax and nn.NLL,
				#while nn.LogSoftmax returns a log value based on e, we want 2 instead
				#m = nn.LogSoftmax(dim=2) -- base e, we want base 2
				m = nn.Softmax(dim=2)
				location_preds_test = m(location_preds_test).detach().to('cpu').numpy()

				#prepare label and mask
				pad_mask_test, label_test = load_label(sp_pos_test, cf, le, 'cpu')

			#	label_test = torch.tensor(label_test, dtype=torch.long, device=device) # changed type
			#	pad_mask_test = torch.tensor(pad_mask_test, dtype=torch.bool, device=device) # changed type

				#compute log likelihood for the batch samples
				res_batch = eval_log_llh(location_preds_test, label_test, pad_mask_test)
				res_llh.append(np.array(res_batch))

				########### Evaluate duration predictions ##########
				# Compute duration prediction error
				duration_labels_test = sp_fix_dur_test[:, :39] # Adjust duration_labels to match the sequence length of duration_preds
				duration_preds_test = duration_preds_test.squeeze(-1) # [259, 39]
				dur_mask_test = mask[:, :39]
				dur_loss = nn.MSELoss(reduction="none")
				duration_error_test = torch.mean(
                    torch.masked_select(dur_loss(duration_preds_test, duration_labels_test), dur_mask_test))
				res_mse.append(duration_error_test.detach().to('cpu').numpy())
			#	print("duration_preds_test", duration_preds_test)
			#	print("duration_labels_test", duration_labels_test)


				if bool(scanpath_gen_flag) == True:
					sn_len = (torch.sum(sn_attention_mask_test, axis=1) - 2).detach().to('cpu').numpy()
					# compute the scan path generated from the model when the first CLS token is given
					sp_dnn, _, dur_dnn = dnn.scanpath_generation(sn_emd=sn_input_ids_test,
														 sn_mask=sn_attention_mask_test,
														 word_ids_sn=None,
														 sn_word_len = sn_word_len_test,
														 le=le,
														 max_pred_len=cf['max_pred_len'])

				#	print("sp_fix_dur_test", sp_fix_dur_test, sp_fix_dur_test.shape)
				#	print("dur_dnn before", dur_dnn, dur_dnn.shape)

					sp_dnn, sp_human = prepare_scanpath(sp_dnn.detach().to('cpu').numpy(),
                                                        dur_dnn.detach().to('cpu').numpy(), sn_len, sp_pos_test,
                                                        sp_fix_dur_test, cf, sn_ids_test, fix_dur_mean, fix_dur_std)

					sp_dnn_list.extend(sp_dnn)
					sp_human_list.extend(sp_human)

				#	print("sp_human", sp_human)
				#	print("sp_dnn", sp_dnn)



				batch_indx +=1

		res_llh = np.concatenate(res_llh).ravel()
	#	res_mse = np.concatenate(res_mse).ravel()
		loss_dict['test_ll'].append(res_llh)
	#	loss_dict['test_mse'].append(res_mse)
		loss_dict['fix_dur_mean'] = fix_dur_mean
		loss_dict['fix_dur_std'] = fix_dur_std
		loss_dict['landing_pos_mean'] = landing_pos_mean
		loss_dict['landing_pos_std'] = landing_pos_std
		loss_dict['sn_word_len_mean'] = sn_word_len_mean
		loss_dict['sn_word_len_std'] = sn_word_len_std

		print('\nTest likelihood is {} \n'.format(np.mean(res_llh)))
		print('\nTest MSE is {} \n'.format(np.mean(res_mse)))
		#save results
		with open('{}/res_BSC_{}_eyettention_{}_Fold{}.pickle'.format(save_data_folder, test_mode, atten_type, fold_indx), 'wb') as handle:
			pickle.dump(loss_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
		fold_indx += 1

	if bool(scanpath_gen_flag) == True:
		#save results
		dic = {"sp_dnn": sp_dnn_list, "sp_human": sp_human_list}
		with open(os.path.join(save_data_folder, f'BSC_scanpath_generation_eyettention_{test_mode}_{atten_type}.pickle'), 'wb') as handle:
			pickle.dump(dic, handle, protocol=pickle.HIGHEST_PROTOCOL)



/usr/local/lib/python3.10/dist-packages/torch/__init__.py:696: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)


Start evaluating on new readers.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


keeping Bert with pre-trained weights
Start training
episode: 0
counter: 1
Sample 1	Location Loss: 3.9333782196	Duration Loss: 0.8744047284 counter: 2
Sample 2	Location Loss: 3.9198908806	Duration Loss: 0.9177789688 counter: 3
Sample 3	Location Loss: 3.9057834148	Duration Loss: 0.9933326840 counter: 4
Sample 4	Location Loss: 3.8889143467	Duration Loss: 0.9780771136 counter: 5
Sample 5	Location Loss: 3.8682518005	Duration Loss: 0.9975040555 counter: 6
Sample 6	Location Loss: 3.8402473927	Duration Loss: 0.9693439603 counter: 7
Sample 7	Location Loss: 3.7982618809	Duration Loss: 0.9542052150 counter: 8
Sample 8	Location Loss: 3.7316331863	Duration Loss: 0.9741965532 counter: 9
Sample 9	Location Loss: 3.6278028488	Duration Loss: 0.9716778994 counter: 10
Sample 10	Location Loss: 3.4897747040	Duration Loss: 0.9688495398 counter: 11
Sample 11	Location Loss: 3.3857207298	Duration Loss: 0.9808064103 counter: 12
Sample 12	Location Loss: 3.2923610210	Duration Loss: 0.9756649137 counter: 13
Sample

/content/utils.py:372: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = torch.tensor(item)


...

duration output tensor([[ 0.0000, -0.6565, -0.0884,  ..., -0.1087, -0.1087, -0.1087],
        [ 0.0000, -0.6565, -0.0884,  ..., -0.1089, -0.1090, -0.1090],
        [ 0.0000, -0.6565, -0.0884,  ..., -0.1087, -0.1087, -0.1087],
        ...,
        [ 0.0000, -0.6565, -0.0816,  ..., -0.0560, -0.0560, -0.0560],
        [ 0.0000, -0.6565, -0.0816,  ..., -0.0560, -0.0560, -0.0616],
        [ 0.0000, -0.6565, -0.0816,  ..., -0.0560, -0.0560, -0.0560]],
       device='cuda:0') torch.Size([256, 60])
location output tensor([[ 0,  1,  2,  ..., 23, 23, 23],
        [ 0,  1,  2,  ..., 23, 23, 23],
        [ 0,  1,  4,  ..., 23, 23, 23],
        ...,
        [ 0,  1,  5,  ..., 19, 19, 19],
        [ 0,  1,  4,  ..., 19, 18, 19],
        [ 0,  1,  2,  ..., 19, 19, 19]], device='cuda:0') torch.Size([256, 60])
generated durations [array([ 0.        , -0.6564644 , -0.08835176, -0.07258851, -0.03651992,
       -0.07030808, -0.03365561,  0.02397131, -0.02972746, -0.11511502,
       -0.05842838, -0.0354133

/content/utils.py:372: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = torch.tensor(item)


duration output tensor([[ 0.0000, -0.6565, -0.0816,  ..., -0.0560, -0.0560, -0.0560],
        [ 0.0000, -0.6565, -0.0816,  ..., -0.0560, -0.0560, -0.0560],
        [ 0.0000, -0.6565, -0.0816,  ..., -0.0513, -0.0878, -0.0650],
        ...,
        [ 0.0000, -0.6565, -0.0884,  ..., -0.1140, -0.1140, -0.1140],
        [ 0.0000, -0.6565, -0.0884,  ..., -0.1140, -0.1140, -0.1140],
        [ 0.0000, -0.6565, -0.0884,  ..., -0.1140, -0.1140, -0.1140]],
       device='cuda:0') torch.Size([256, 60])
location output tensor([[ 0,  1,  3,  ..., 19, 19, 19],
        [ 0,  1,  2,  ..., 19, 19, 19],
        [ 0,  1,  3,  ..., 19, 19, 19],
        ...,
        [ 0,  1,  4,  ..., 24, 24, 24],
        [ 0,  1,  2,  ..., 24, 24, 24],
        [ 0,  1,  3,  ..., 24, 24, 24]], device='cuda:0') torch.Size([256, 60])
generated durations [array([ 0.        , -0.6564644 , -0.08158398, -0.02310366, -0.04715646,
        0.01070034, -0.07419597, -0.04143744, -0.06823965], dtype=float32), array([ 0.0000000e+00, -6.

/content/utils.py:372: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = torch.tensor(item)


KeyboardInterrupt: 